In [2]:
from __future__ import annotations

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{Boring Object Orientation}
\author{Moshe Zadka -- https://cobordism.com}
\date{2021}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## OO Python

### Python and object  programming

```{=latex}
\begin{frame}[fragile]
\frametitle{Python and object oriented programming}

Everything is an object
\end{frame}
```

### Why OO design principles

```{=latex}
\begin{frame}[fragile]
\frametitle{Why OO design principles?}

Guidelines to code that is easy to maintain
\end{frame}
```

### Do OO design principles work?

```{=latex}
\begin{frame}[fragile]
\frametitle{Do OO design principles work?}

Yes
\pause
...but
\end{frame}
```


### Principles

```{=latex}
\begin{frame}[fragile]
\frametitle{Principles}

Make your objects more boring!
The simple tricks that they don't want you to know!

\begin{itemize}
\item Declare interfaces\pause 
\item Simplify initialization\pause
\item Avoid mutation\pause
\item Avoid hiding\pause
\item Avoid methods\pause
\item Avoid inheritance
\end{itemize}

\end{frame}


## Interfaces

### Why declare interfaces

```{=latex}

\begin{frame}[fragile]
\frametitle{Why declare interfaces?}

Explicit is better than implicit
\end{frame}
```

### Declaring interfaces with zope.interface

```{=latex}
\begin{frame}[fragile]
\frametitle{Declaring interfaces with zope.interface}
```

In [3]:
from zope import interface

class ISprite(interface.Interface):

    bounding_box = interface.Attribute(
        "The bounding box"
    )

    def intersects(box):
        "Does this intersect with a box"

```{=latex}
\end{frame}
```

### Testing for interface provision

```{=latex}
\begin{frame}[fragile]
\frametitle{Testing for interface provision}
```

In [51]:
from zope.interface import verify

def test_implementation():
    sprite = make_sprite()
    verify.verifyObject(ISprite, sprite)

```{=latex}
\end{frame}
```

## Boring constructors

        
### Interesting constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Interesting constructors}
```

In [52]:
class Stuff:

    def __init__(self, fname):
        # Create a new object
        self.destination = Destination()
        # Call a system call
        self.finput = open(fname)

```{=latex}
\end{frame}
```

### Boring constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Boring constructors}
```

In [53]:
class Stuff:

    def __init__(self, finput, destination):
        self.destination = destination
        self.finput = finput

    @classmethod
    def from_name(cls, name):
        # Create a new object
        destination = Destination()
        # Call a system call
        finput = open(fname)
        return cls(finput, destination)

```{=latex}
\end{frame}
```

### Why boring constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Why boring constructors?}

\begin{itemize}
\item Testing\pause 
\item Correctness\pause
\item Async\pause
\end{itemize}

\end{frame}
```

### Using attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Using attrs}
```

In [54]:
import attr

@attr.s(auto_attribs=True)
class Stuff:
    finput: Any
    destination: Any

```{=latex}
\end{frame}
```

## Immutability

### Immutable objects

```{=latex}
\begin{frame}[fragile]
\frametitle{Immutable objects}
```

In [55]:
@attr.s(auto_attribs=True, frozen=True)
class Stuff:
    destination: Any
    finput: Any

In [56]:
import io

class Destination:
    pass

In [57]:
my_stuff = Stuff(Destination(), io.StringIO())

In [58]:
try:
    my_stuff.finput = io.StringIO()
except Exception as exc:
    print(repr(exc))

FrozenInstanceError()


```{=latex}
\end{frame}
```

### Immutablity as bug avoidance

```{=latex}
\begin{frame}[fragile]
\frametitle{Immutablity as bug avoidance}
```

In [59]:
def some_function(some_list=[]):
    pass

```{=latex}
\end{frame}
```

### Immutablity as interface simplifying

```{=latex}
\begin{frame}[fragile]
\frametitle{Immutablity as interface simplifying}

No variation, no invariant breakage!
\end{frame}
```

### Frozen attrs

```{=latex}
\begin{frame}[fragile]
\frametitle{Change without mutation}
```

In [60]:
@attr.s(auto_attribs=True, frozen=True)
class Point:
    x: float
    y: float

In [61]:
origin = Point(0, 0)

In [62]:
up = attr.evolve(origin, y=1)

In [63]:
origin, up

(Point(x=0, y=0), Point(x=0, y=1))

```{=latex}
\end{frame}
```

## Methods

### Private methods issues

```{=latex}
\begin{frame}[fragile]
\frametitle{Private methods}
```

In [64]:
class HTTPSession:
    def _request(self, method, url):
        pass
    def get(self, url):
        return self._request('GET', url)
    def head(self, url):
        return self._request('HEAD', url)

```{=latex}
\end{frame}
```

### Refactoring private methods away

```{=latex}
\begin{frame}[fragile]
\frametitle{Private methods to separate class}
```

In [65]:
class RawHTTPSession:
    def request(self, method, url):
        pass

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Private attribute instance}
```

In [66]:
class HTTPSession:
    _raw: RawHTTPSession
    def get(self, url):
        return self._raw.request('GET', url)
    def head(self, url):
        return self._raw.request('HEAD', url)

```{=latex}
\end{frame}
```

### Polyomorphism

```{=latex}
\begin{frame}[fragile]
\frametitle{Methods}
```

In [67]:
@attr.s(auto_attribs=True, frozen=True)
class Point2D:
    x: float
    y: float

    def distance_from_origin(self):
        return (self.x**2 + self.y**2) ** 0.5

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Methods}
```

In [68]:
@attr.s(auto_attribs=True, frozen=True)
class Point3D:
    x: float
    y: float
    z: float

    def distance_from_origin(self):
        return (self.x**2 +
                self.y**2 +
                self.z**2)

```{=latex}
\end{frame}
```

### Bloating

```{=latex}

\begin{frame}[fragile]
\frametitle{Why not methods?}

Bloats classes

\end{frame}
```

### Singledispatch

```{=latex}
\begin{frame}[fragile]
\frametitle{Methodless Polymorphism}
```

In [69]:
@attr.s(auto_attribs=True, frozen=True)
class Point2D:
    x: float;
    y: float

In [70]:
@attr.s(auto_attribs=True, frozen=True)
class Point3D:
    x: float
    y: float
    z: float

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Methodless Polymorphism}
```

In [71]:
import functools

In [72]:
@functools.singledispatch
def distance_from_origin(pt):
    raise NotImplementedError(point)

In [73]:
@distance_from_origin.register(Point2D)
def distance_2d(pt):
    return (pt.x**2 + pt.y**2) ** 0.5

In [74]:
@distance_from_origin.register(Point3D)
def distance_3d(pt):
    return (pt.x**2 + pt.y**2 + pt.z**2) ** 0.5

```{=latex}
\end{frame}
```

## Inheritance-as-API

In [75]:
from unittest import mock
Kernel = generic = basic = mock.MagicMock()

### Examples in the wild: Twisted

```{=latex}
\begin{frame}[fragile]
\frametitle{Inheritance as API: Twisted}
```

In [76]:
# From the Twisted tutorial
class FingerProtocol(basic.LineReceiver):
    def lineReceived(self, user):
        self.transport.write(b"No such user\r\n")
        self.transport.loseConnection()

```{=latex}
\end{frame}
```

### Examples in the wild: Django

```{=latex}
\begin{frame}[fragile]
\frametitle{Inheritance as API: Django}
```

In [77]:
# From the Django tutorial
class IndexView(generic.ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]

```{=latex}
\end{frame}
```

### Examples in the wild: Jupyter

```{=latex}
\begin{frame}[fragile]
\frametitle{Inheritance as API: Jupyter}
```

In [78]:
# From the Jupyter documentation

class EchoKernel(Kernel):
    implementation = 'Echo'
    implementation_version = '1.0'
    language = 'no-op'
    language_version = '0.1'
    language_info = {
        'name': 'Any text',
        'mimetype': 'text/plain',
        'file_extension': '.txt',
    }
    banner = "Echo kernel - as useful as a parrot"

    def do_execute(self, code, silent, store_history=True, user_expressions=None,
                   allow_stdin=False):
        ...

```{=latex}
\end{frame}
```

### Issues

```{=latex}
\begin{frame}[fragile]
\frametitle{Inheritance as API: Issues}

"Shared everything"
\end{frame}
```

### Composition

```{=latex}
\begin{frame}[fragile]
\frametitle{Composition}

\begin{itemize}
\item Define {\em interface}
\item Useful behavior in {\em referred class}
\end{itemize}

\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Composition: Example}
```

In [79]:
class IMovable(interface.Interface):
    x_position = interface.Attribute("x coordinate")
    y_position = interface.Attribute("y coordinate")
    def tick():
        pass

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Composition: Example}
```

In [80]:
@interface.implementer(IMovable)
@attr.s(auto_attribs=True)
class StraightLine:
    dx: float
    dy: float
    x_position: float
    y_position: float
    def tick(self):
        self.x_position += dx
        self.y_position += dy

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Composition: Example}
```

In [81]:
@interface.implementer(IMovable)
@attr.s(auto_attribs=True)
class Sprite:
     _movable: IMovable
     @property
     def x_position(self):
         return self._movable.x_position
     @property
     def y_position(self):
         return self._movable.y_position
     def tick(self):
         return self._movable.tick()

```{=latex}
\end{frame}
```

## Wrap-up

### No rules

```{=latex}
\begin{frame}[fragile]
\frametitle{Python: Language of the free}

Diamond inheritance with overriddable constructors as mandatory interface?
Sure!
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Python: With Great Power}

Diamond inheritance with overriddable constructors as mandatory interface?

Maybe not...
\end{frame}
```

### Opaque lessons

```{=latex}
\begin{frame}[fragile]
\frametitle{Opaque lessons learned}

\begin{itemize}
\item Do as we say, not as we do
\item Big systems, big headaches
\end{itemize}
\end{frame}
```

### Principles

```{=latex}
\begin{frame}[fragile]
\frametitle{Less interesting code}

Be dumb as possible when writing code.

\end{frame}
```

```{=latex}
\end{document}
```